# Force Directed Graph Sim
#### Authors: Shakeel Khan

First, our imports.

In [ ]:
from Graph import Graph, Node
from math import log, pow
import random
import numpy as np
import matplotlib.pyplot as plt

Define the constants. These are the constants for the sim:

In [ ]:
# Sim constant definitions.
NUM_STEPS = 250
NUM_NODES = 20        # Don't exceed 30.
NODES_PER_GROUP = 5

And these are used in the calculations for the attractive and repelling forces between the nodes. There is a comment for each of them describing how I interpeted them. This is because the paper itself just calls them C1-C4, and goes "here is a reccomended set of values" with no further explanation, and I figured it would be beneficial to understand the role these constants play in the calculation of these forces.

In [ ]:
C1 = 2.0    # This is the strength of the attractive force.
C2 = 10.0   # This is the distance desired between nodes
C3 = 5.0    # This is the strength of the repelling force.
C4 = 1.0    # This is the effective speed of the nodes.

Now we define our graph, create the nodes, and the edges in between.

Here's a quick rundown for those of you who'd like to tinker with it.

# Working With the `Graph` Class
## Creating an Instance of the Class


In [ ]:
# Now we define our graph.
graph = Graph()
nodes = []

# This is used to make sure the starting position of each node is unique.
def isUniquePos(pos):
    for node in nodes:
        if np.array_equal(pos, node.position):
            return False
    return True

# Create our nodes.
random.seed(a = 10)
for i in range(NUM_NODES):
    # Keep generating random positions till we get something unique.
    x = random.randrange(320, 340, 1)
    y = random.randrange(240, 260, 1)

    pos = np.array([x, y], dtype=np.double)
    while (not isUniquePos(pos)):
        pos[0] = random.randrange(320, 340, 1)
        pos[1] = random.randrange(240, 260, 1)

    # Create the label for the node.
    label = str(i)

    # If this is the 'leader' of a group (it's directly connected to all the
    # nodes in the group).
    if (i % NODES_PER_GROUP == 0):
        label += '***'

    # Create the node, add it to our list and add it to the graph.
    node = Node(label, pos)
    nodes.append(node)
    graph.addNode(node)

# Create the edges.
# TODO: Clean this up to model groups which actually would appear in ProVis.
#       (Having one node sticking out from the "leader" of the group which
#        used to connect to the other groups).
# TODO: Try to add a way to add other disjoint groups.
for i in range(0, NUM_NODES, NODES_PER_GROUP):
    if (i != 0 and i != NUM_NODES):
        graph.addEdge(nodes[i - NODES_PER_GROUP].id, nodes[i].id)
        print('***(' + str(i - NODES_PER_GROUP) + ', ' + str(i) + ')')
    stop = i + NODES_PER_GROUP
    if (stop > NUM_NODES):
        stop = NUM_NODES - i
    for j in range (i + 1, stop):
        graph.addEdge(nodes[i].id, nodes[j].id)
        print('(' + str(i) + ', ' + str(j) + ')')

# This was originally here for debugging purposes when implementing the graph,
# and is no longer necessary.
# graph.print()